**Имеющаяся ситуации:** на данный момент имеются рабочие модели, которые успешно классифицируют запросы пользователей по таким темам как "Обещанный платеж", "Куда делись деньги", "Как подключить интернет", "Хочу поменять тариф", и т.д.

**Задача:** Углубить обработку запросов пользователей в чат-боте технической поддержки и категоризировать их по заданным, более узким в отличии от уже работающих, критериям.

Далее представлены конкретные задачи от заказчика.

---

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg .tg-lboi{border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-0lax{text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 759px">
<colgroup>
<col style="width: 27px">
<col style="width: 125px">
<col style="width: 105px">
<col style="width: 171px">
<col style="width: 331px">
</colgroup>
  <tr>
    <th class="tg-lboi">№</th>
    <th class="tg-lboi">Запрос пользователя</th>
    <th class="tg-0lax">Результат текущего фреймворка</th>
    <th class="tg-0lax">Требуемый результат</th>
    <th class="tg-0lax"><center>Комментарий</th>
  </tr>
  <tr>
    <td class="tg-lboi">1</td>
    <td class="tg-lboi">Подключен ли у меня мобильный интернет</td>
    <td class="tg-0lax">Как подключить интернет</td>
    <td class="tg-0lax">Классифицировать как "подключен ли мобильный интернет"</td>
    <td class="tg-0lax">Создадим новый классификатор "Подключен ли \ Есть ли" на том же уровне что и классификатор "Хочу отключить услугу/опцию/подписку". Частицу "ли" будем добавлять к слову перед ней. Модель будет обрабатывать запросы на получение информации.<br>Далее запрос будет классифицироваться на наличие в нем названия услуги</td>
  </tr>
  <tr>
    <td class="tg-lboi">2</td>
    <td class="tg-lboi">Не ноступен обещенный платеж. Почему</td>
    <td class="tg-0lax">Обещанный платеж</td>
    <td class="tg-0lax">Из фразы понятен контекст: "Почему не доступен ОП"</td>
    <td class="tg-0lax">Решение: Не менять модель, а просто после классификации в "Обещанный платеж" проверять содержание сообщения на предмет слов "Почему", "не доступен" и подобных.</td>
  </tr>
  <tr>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">За что сегодня снялись 15р?</td>
    <td class="tg-0lax">Куда делись деньги</td>
    <td class="tg-0lax">"Начисления за подписка: 21.10.2019 - 15р"</td>
    <td class="tg-0lax">Простое решение, которое приходит в голову - после классификации искать в тексте исходного запроса слова "сегодня", "вчера" и даты, а так же суммы. Можно так же искать и только суммы.</td>
  </tr>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">Я не буду платить за обещанный платеж</td>
    <td class="tg-0lax">Обещанный платеж</td>
    <td class="tg-0lax">Распознать: "отказывается от факта", "хочет отключить"</td>
    <td class="tg-0lax">Случаи с "не хочу", "не буду" классифицировать как "Хочу отключить услугу" - переобучить существующую модель</td>
  </tr>
</table>

**Так как реальных данных нет, работу я решил продемонстрировать на маленьком датасете состоящим из смс-сообщений, размеченном на spam/ham. Далее работа будет вестись с ним.**

План исследования:
1. Тест классических линейных алгоритмов в sklearn
2. Vowpal Wabbit. Использует линейные алгоритмы. Удобна для работы с текстом, подходит для онлайн-обучения.
3. Fasttext. Под капотом нейронная сеть с софтмаксом в качестве функции потерь. Встроенный метод разбиения на буквенные n-граммы.

### 1. Тест классических линейных алгоритмов в sklearn

Используем для обучения метод мешка слов созданный по методу TF-IDF. Результатом будет являться матрица объекты-признаки с весами, учитывающих как частоту слова в выборке, так и частоту слова в разных текстах (обратная частота).

Лемматизацию проводить не будем, так как очевидно, что преведение слов к их начальным формам помешает отличать разные категории запросов с участием одних и тех же слов.

В качестве тестритуемых моделей будем проверять логистическую регрессию, линейный классификатор, линейный SVM, а так же наивный баессовский классификатор. Все они хорошо показывают себя в задачах классификации текстов. Они хорошо масштабируются, могут работать с большим количеством признаков, на очень больших выборках.

In [1]:
import warnings
import sys
import re
import string
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics

warnings.filterwarnings("ignore")

Загрузим данные. Датасет состоит из 5575 смс-сообщений, 744 из них помечены как спам.

In [2]:
with open('SMSSpamCollection.txt', encoding='UTF-8') as f:
    texts = f.read() 

In [3]:
# Выполним предобработку. Приведем к нижнему регистру, удалим цифры, пунктуацию.

texts = texts.lower()
texts = re.sub(r'\d+', '', texts)
texts = texts.translate(str.maketrans('', '', string.punctuation))

# Получим метки

texts = texts.split('\n')
labels = [0 if 'ham' in i else 1 for i in texts]

# Уберем метки из текстов

texts = [i.replace('ham', '') for i in texts]
texts = [i.replace('spam', '') for i in texts]
texts = [i.replace('\t', '') for i in texts]

In [4]:
# Разобъем выборку на обучение и тест

train_documents, test_documents, train_labels, test_labels = train_test_split(texts, 
                                                                              labels, random_state=17)

#### Оценка качества работы разных классификаторов

In [5]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [6]:
# Тестируем классификаторы, оцениваем на кросс-валидации с помощью площади под ROC-AUC кривой

roc_auc = metrics.make_scorer(metrics.roc_auc_score)

for clf in [LogisticRegression, LinearSVC, SGDClassifier, MultinomialNB]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), 
                                          TfidfTransformer(), 
                                          clf()), train_documents, 
                                          train_labels, scoring=roc_auc).mean())
    print("\n")

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8382421063779119


<class 'sklearn.svm.classes.LinearSVC'>
0.9470760424800648


<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.9517620670774916


<class 'sklearn.naive_bayes.MultinomialNB'>
0.7927382830392253




#### Оценка использования n-грамм

Далее применим n-граммы и проверим вариацию качества на кросс-валидации. Потестим на только биграммах, затем униграммах и биграммах, затем униграммах, биграммах и триграммах.

Кроме того, учитывая наличие большого количества опечаток в текстах, в дальнейшем полезно попробовать применить буквенные n-граммы, которые могут помочь улучшить классификацию текстов с опечатками. Данный метод встроен в fasttext.

In [7]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier, MultinomialNB]:
    print(clf)
    for i in [[2,2], [1,2], [1,3]]:
        print('Ngram range:', i)
        res = cross_val_score(text_classifier(CountVectorizer(ngram_range=(i)), TfidfTransformer(), 
                                              clf()), train_documents, 
                                              train_labels, scoring=roc_auc, cv=3).mean()
        print('Cross-val-score roc-auc score: %.4f\n' % res)
    

<class 'sklearn.linear_model.logistic.LogisticRegression'>
Ngram range: [2, 2]
Cross-val-score roc-auc score: 0.5924

Ngram range: [1, 2]
Cross-val-score roc-auc score: 0.7926

Ngram range: [1, 3]
Cross-val-score roc-auc score: 0.7463

<class 'sklearn.svm.classes.LinearSVC'>
Ngram range: [2, 2]
Cross-val-score roc-auc score: 0.8660

Ngram range: [1, 2]
Cross-val-score roc-auc score: 0.9491

Ngram range: [1, 3]
Cross-val-score roc-auc score: 0.9506

<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
Ngram range: [2, 2]
Cross-val-score roc-auc score: 0.8959

Ngram range: [1, 2]
Cross-val-score roc-auc score: 0.9577

Ngram range: [1, 3]
Cross-val-score roc-auc score: 0.9635

<class 'sklearn.naive_bayes.MultinomialNB'>
Ngram range: [2, 2]
Cross-val-score roc-auc score: 0.7455

Ngram range: [1, 2]
Cross-val-score roc-auc score: 0.7403

Ngram range: [1, 3]
Cross-val-score roc-auc score: 0.7141



Далее провалидируем лучшую модель на отложенной выборке.

In [8]:
# Посчитаем площадь под ROC-AUC кривой для SGD и SVC классификаторов на отложенной выборке

SGD_clf = text_classifier(CountVectorizer(ngram_range=([1, 3])), TfidfTransformer(), SGDClassifier())
SGD_clf.fit(train_documents, train_labels)
print('SGDclassificator ROC-AUC: ' + str(metrics.roc_auc_score(SGD_clf.predict(test_documents), test_labels)))

svc_clf = text_classifier(CountVectorizer(ngram_range=([1, 3])), TfidfTransformer(), LinearSVC())
svc_clf.fit(train_documents, train_labels)
print('SVC classificator ROC-AUC: ' + str(metrics.roc_auc_score(svc_clf.predict(test_documents), test_labels)))

SGDclassificator ROC-AUC: 0.9774868314424635
SVC classificator ROC-AUC: 0.9795812758420053


#### Vowpal Wabbit

Воспользуемся библиотекой Vowpal Wabbit которая широко используется для работы с текстом и онлайн обучения моделей. В ней так же используются линейные модели.

Напишем функцию, которая будет приводить наши данные к формату, который переваривает Vowpal Wabbit.

In [9]:
def to_vw_format(document, label=None):
    '''Brings sample to vowpal wabbit format'''
    
    return str(label or '') + ' |text ' + document + '\n'

to_vw_format(train_documents[0], 1 if labels == 1 else -1)

'-1 |text discussed with your mother ah\n'

In [10]:
# Приведем выборку к формату VW

with open('train.vw', 'w', encoding="utf-8") as vw_train_data:
    for text, target in zip(train_documents, train_labels):
        try:
            vw_train_data.write(to_vw_format(text, 1 if target == 1 else -1))
        except:
            err = to_vw_format(text, target)
            print(err)
            break
with open('test.vw', 'w', encoding="utf-8") as vw_test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))

Запустим Vowpal Wabbit на сформированном файле. Зададим функцию потерь, которая показала лучшее значение на предыдущих тестах. Добавим n-граммы. Построенную модель сохраним в соответствующий файл model.vw

In [11]:
!vw -d train.vw --loss_function classic --ngram 2 --ngram 3 -f model.vw

Generating 2-grams for all namespaces.
Generating 3-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0  -1.0000   0.0000       13
1.148252 1.296504            2            2.0   1.0000  -0.1386       34
0.999481 0.850709            4            4.0   1.0000   0.0945       55
0.775353 0.551225            8            8.0  -1.0000  -0.4509       25
0.729443 0.683533           16           16.0  -1.0000  -0.9039       34
0.694735 0.660028           32           32.0   1.0000  -0.1515       61
0.453237 0.211738           64           64.0  -1.0000  -0.7939       37
0.406101 0.358964          128          128.0  -1.0000  -1.0000       55
0.298645 0.191189          256       

Применим обученную модель на тестовой выборке, сохраним результат в файл

In [12]:
!vw -i model.vw -t -d test.vw -p test_predictions.txt --binary

Generating 2-grams for all namespaces.
Generating 3-grams for all namespaces.
only testing
predictions = test_predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
    n.a.     n.a.            1            1.0  unknown   1.0000       43
    n.a.     n.a.            2            2.0  unknown  -1.0000       31
    n.a.     n.a.            4            4.0  unknown  -1.0000       28
    n.a.     n.a.            8            8.0  unknown  -1.0000       34
    n.a.     n.a.           16           16.0  unknown  -1.0000       58
    n.a.     n.a.           32           32.0  unknown  -1.0000       46
    n.a.     n.a.           64           64.0  unknown   1.0000       73
    n.a.     n.a.          128          128.0  unknown  -1.0000       25
    n.a.     n.a.

Загрузим полученные предсказания, вычислим AUC

In [13]:
test_prediction = None
with open('test_predictions.txt') as pred_file:
    test_prediction = [float(label) 
                             for label in pred_file.readlines()]

auc = metrics.roc_auc_score(test_labels, test_prediction)
auc

0.9464469226963043

In [14]:
test_l = list(map(lambda x: 0 if x == -1 else 1, test_prediction))

In [15]:
metrics.precision_score(test_labels, test_l)

0.9808917197452229

In [16]:
metrics.recall_score(test_labels, test_l)

0.8953488372093024

VW так же удобно использовать при многоклассовой классификации, для этого при обучении на соответсвующей выборке (классы должны быть нумерованы начиная с 1) передается параметр --oaa ('one against all)

Настраиваемые гиперпараметры, которые оказывают существенное влияние на качество модели:


- темп обучения (-l, по умолчанию 0.5) – коэффициент перед изменением весов модели при каждом изменении
- степень убывания темпа обучения (--power_t, по умолчанию 0.5) – на практике проверено, что если темп обучения уменьшается при увеличении числа итераций стохастического градиентного спуска, то минимум функции находится лучше
- функция потерь (--loss_function) – от нее, по сути, зависит обучаемый алгоритм. Про функции mпотерь в документации
- регуляризация (-l1) – тут надо обратить внимание на то, что в VW регуляризация считается для каждого объекта, поэтому коэффициенты регуляризации обычно берутся малыми, около  10−20.

Для обучения классификатора воспользуемся библиотекой fasttext, в основе лежит использование буквенных n-грамм, что позволяет алгоритму успешно работать с текстами, содержащими ошибки, а так же незнакомые слова.

In [17]:
import fasttext

In [18]:
with open('SMSSpamCollection.txt', encoding='UTF-8') as f:
    texts = f.read() 

Для улучшения классификации следует применять простой предпроцессинг вроде приведения к нижнему регистру, удаление знаков препинания.

In [19]:
# Выполним предобработку. Приведем к нижнему регистру, удалим цифры, пунктуацию.

texts = texts.lower()
texts = re.sub(r'\d+', '', texts)
texts = texts.translate(str.maketrans('', '', string.punctuation))

# Добавим метки в соответствии с требованиями fasttext

texts = texts.replace('ham', '__label__ham ')
texts = texts.replace('spam', '__label__spam ')
texts = texts.replace('\t', '')
texts = texts.split('\n')

In [20]:
# Разобъем выборку на обучение и тест

train_documents, test_documents = train_test_split(texts, random_state=17)

In [21]:
with open('train_documens.txt', 'w', encoding='UTF-8') as f:
    for i in train_documents:
        f.write(i + '\n')
#         f.writelines(train_documents)
with open('test_documens.txt', 'w', encoding='UTF-8') as f:
    for i in test_documents:
        f.write(i + '\n')

In [22]:
# Обучим модель

model = fasttext.train_supervised(input='train_documens.txt')

In [23]:
# Получим значения точности, полноты на отложенной выборке:

model.test("test_documens.txt")

(1394, 0.9813486370157819, 0.9813486370157819)

In [24]:
# Выведем предсказание для рандомного текста


model.predict("Why not put knives in the dishwasher?", k=2)

(('__label__ham', '__label__spam'), array([1.00000858e+00, 1.13843689e-05]))

Настраиваевым гиперпараметром модели является число итераций по выборке epochs, которое по-умолчанию равно 5, что в зависимости от размера выборки может быть недостаточно.


Еще одним гиперпараметром является learning rate или шаг обучения, которая влияет на скорость обучения и принимает значения 0.1 - 1. 0 - модель не меняется вообще. 

Увеличим количество итераций до 11, выставим learning rate 0.5. Данные параметры можно понастраивать в paramgrid где можно перебирать значения гиперпараметров с заданным шагом, а затем сравнивать качество алгоритмов.

In [25]:
model = fasttext.train_supervised(input="train_documens.txt", epoch=11, lr=0.5)

In [26]:
model.test("test_documens.txt")

(1394, 0.9813486370157819, 0.9813486370157819)

Fasttext также позволяет применять n-граммы

In [27]:
model = fasttext.train_supervised(input="train_documens.txt", epoch=11, lr=0.1, wordNgrams=3)

In [28]:
model.test("test_documens.txt")

(1394, 0.9799139167862266, 0.9799139167862266)

Алгоритмы sklearn заметно уступают VW и fasttext.

Лучшие значения метрик на отложенной выборке показывает fasttest. Хотя конечно же не достаточно, что бы однозначно говорить, что fasttest VW действительно лучше работает на данном распределении чем алгоритм fasttext.